## 导入必要的库

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

## 读取数据

根据这个UCI官网上下载的数据集，我们发现有两个数据集，分别是关于红酒质量的和关于白酒质量的，我们将其读入数据中。

In [38]:
pip install ucimlrepo

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from ucimlrepo import fetch_ucirepo
wine_quality = fetch_ucirepo(id=186)
# data (as pandas dataframes) 
X = wine_quality.data.features 
y = wine_quality.data.targets 
  
# 将特征和目标变量合并为一个DataFrame
wine_data = pd.concat([X, pd.DataFrame(y, columns=['quality'])], axis=1)

查看导入的数据集。

查看数据集的基本信息。

In [3]:
wine_data.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


## 数据清洗（Data Cleaning）

### 缺失值处理（Missing Value Handling）

In [4]:
# 计算每列缺失值的数量
missing_values = wine_data.isna().sum()
# 打印每列缺失值的数量
print("每列缺失值的数量：")
print(missing_values)

每列缺失值的数量：
fixed_acidity           0
volatile_acidity        0
citric_acid             0
residual_sugar          0
chlorides               0
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64


In [5]:
wine_data.dropna(inplace=True)

### 重复值处理（Duplicate Data Handling）

In [6]:
# 2. 处理重复值
wine_data.drop_duplicates(inplace=True)

## 数据集成（Data Integration）
合并来自不同来源的具有相同属性的数据。对于这次的作业，我们将计算“总酸度（total_acidity）”，即“固定酸度（fixed_acidity）”和“挥发性酸度（volatile_acidity）”的总和，并将其作为新列添加到数据集中。

In [7]:
# 计算总酸度并添加为新列
wine_data['total_acidity'] = wine_data['fixed_acidity'] + wine_data['volatile_acidity']

In [8]:
print("数据的前几行：")
print(wine_data.head())

数据的前几行：
   fixed_acidity  volatile_acidity  citric_acid  residual_sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
5            7.4              0.66         0.00             1.8      0.075   

   free_sulfur_dioxide  total_sulfur_dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
5                 13.0                  40.0   0.9978  3.51       0.56   

   alcohol  quality  total_acidity  
0      9.4        5           8.10  
1   

## 数据转换（Data Transformation）

### 标准化（Normalization）

将“质量（quality）”数据标准化到[0,1]范围内。

In [9]:
scaler = MinMaxScaler()
wine_data['quality_normalized'] = scaler.fit_transform(wine_data[['quality']])
wine_data.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,total_acidity,quality_normalized
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,8.10,0.333333
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,8.68,0.333333
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,8.56,0.333333
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,11.48,0.500000
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5,8.06,0.333333


### 离散化（Discretization）

将连续属性“fixed acidity”离散化为三个级别：“低”、“中”和“高”。

In [10]:
# 查看fixed acidity的最小值和最低值，以及三分之一数量的分界点和三分之二分界点

# 使用describe()函数查看统计信息
fixed_acidity_stats = X['fixed_acidity'].describe()

# 输出结果
print("最小值:", fixed_acidity_stats['min'])
print("最大值:", fixed_acidity_stats['max'])
print("三分之一分位点:", X['fixed_acidity'].quantile(1/3))
print("三分之二分位点:", X['fixed_acidity'].quantile(2/3))

最小值: 3.8
最大值: 15.9
三分之一分位点: 6.6
三分之二分位点: 7.4


In [11]:
# 2. 离散化
bins = [0, 6.6, 7.4, 16]  # 设置分箱边界
labels = ['low', 'medium', 'high']  # 设置分箱标签
wine_data['fixed_acidity_discretized'] = pd.cut(wine_data['fixed_acidity'], bins=bins, labels=labels)
wine_data.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,total_acidity,quality_normalized,fixed_acidity_discretized
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,8.10,0.333333,medium
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,8.68,0.333333,high
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,8.56,0.333333,high
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,11.48,0.500000,high
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5,8.06,0.333333,medium


In [12]:
# 计算每列缺失值的数量
missing_values = wine_data.isna().sum()
# 打印每列缺失值的数量
print("每列缺失值的数量：")
print(missing_values)

每列缺失值的数量：
fixed_acidity                0
volatile_acidity             0
citric_acid                  0
residual_sugar               0
chlorides                    0
free_sulfur_dioxide          0
total_sulfur_dioxide         0
density                      0
pH                           0
sulphates                    0
alcohol                      0
quality                      0
total_acidity                0
quality_normalized           0
fixed_acidity_discretized    0
dtype: int64


## 数据降维（Data Reduction）

特征选择：使用方差分析（ANOVA）选择对葡萄酒品质评级影响最显著的前三个特征。

In [20]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# 创建恃征选择器
selector = SelectKBest(f_classif, k=3)

# 使用特征选择器来拟合并转换 x
senew = selector.fit_transform(X, y)

# 打印被选择的特征
mask = selector.get_support()
new_features = X.columns [mask]
print(new_features)

# 打印所有特征的F 值
for feature, score in zip(X.columns, selector.scores_):
    print(f"Feature: {feature}, Score: {score}")

Index(['volatile_acidity', 'density', 'alcohol'], dtype='object')
Feature: fixed_acidity, Score: 8.004192723298823
Feature: volatile_acidity, Score: 96.67402209622112
Feature: citric_acid, Score: 9.31019870644853
Feature: residual_sugar, Score: 9.111366826994608
Feature: chlorides, Score: 50.84971882802702
Feature: free_sulfur_dioxide, Score: 14.939170402675012
Feature: total_sulfur_dioxide, Score: 7.716088194433556
Feature: density, Score: 136.95123571943913
Feature: pH, Score: 2.021462044859375
Feature: sulphates, Score: 4.325772856450053
Feature: alcohol, Score: 320.59344780302115


/Users/mac/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
